In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.nn.utils import spectral_norm
import argparse
import numpy as np
import cv2
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
import os
from tqdm import tqdm
import torchvision.models as models
from torch.nn.parallel import DataParallel

### Dataset

In [2]:
class AnimeDataSet(Dataset):
    def __init__(self, args, transform=None):
        """
        folder structure:
            - {data_dir}
                - photo
                    1.jpg, ..., n.jpg
                - {dataset}  # E.g Hayao
                    smooth
                        1.jpg, ..., n.jpg
                    style
                        1.jpg, ..., n.jpg
        """
        data_dir = args['data_dir']
        dataset = args['dataset']
        # data_dir = "data"
        # dataset = "Hayao"

        anime_dir = os.path.join(data_dir, dataset)
        if not os.path.exists(data_dir):
            raise FileNotFoundError(f'Folder {data_dir} does not exist')

        if not os.path.exists(anime_dir):
            raise FileNotFoundError(f'Folder {anime_dir} does not exist')

        self.mean = compute_data_mean(os.path.join(anime_dir, 'style'))
        print(f'Mean(B, G, R) of {dataset} are {self.mean}')

        self.data_dir = data_dir
        self.image_files = {}
        self.photo = 'train_photo'
        self.style = f'{dataset}/style'
        self.smooth = f'{dataset}/smooth'
        self.dummy = torch.zeros(3, 256, 256)

        for opt in [self.photo, self.style, self.smooth]:
            folder = os.path.join(data_dir, opt)
            files = os.listdir(folder)
            self.image_files[opt] = [os.path.join(folder, fi) for fi in files]

        self.transform = transform
        print(f'Dataset: real {len(self.image_files[self.photo])} style {self.len_anime}, smooth {self.len_smooth}')

    def __len__(self):
        return len(self.image_files[self.photo])

    @property
    def len_anime(self):
        return len(self.image_files[self.style])

    @property
    def len_smooth(self):
        return len(self.image_files[self.smooth])

    def __getitem__(self, index):
        image = self.load_photo(index)
        anm_idx = index
        if anm_idx > self.len_anime - 1:
            anm_idx -= self.len_anime * (index // self.len_anime)

        anime, anime_gray = self.load_anime(anm_idx)
        smooth_gray = self.load_anime_smooth(anm_idx)

        return image, anime, anime_gray, smooth_gray

    def load_photo(self, index):
        fpath = self.image_files[self.photo][index]
        image = cv2.imread(fpath)[:,:,::-1]
        image = self._transform(image, addmean=False)
        image = image.transpose(2, 0, 1)
        return torch.tensor(image)

    def load_anime(self, index):
        fpath = self.image_files[self.style][index]
        image = cv2.imread(fpath)[:,:,::-1]

        image_gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
        image_gray = np.stack([image_gray, image_gray, image_gray], axis=-1)
        image_gray = self._transform(image_gray, addmean=False)
        image_gray = image_gray.transpose(2, 0, 1)

        image = self._transform(image, addmean=True)
        image = image.transpose(2, 0, 1)

        return torch.tensor(image), torch.tensor(image_gray)

    def load_anime_smooth(self, index):
        fpath = self.image_files[self.smooth][index]
        image = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
        image = np.stack([image, image, image], axis=-1)
        image = self._transform(image, addmean=False)
        image = image.transpose(2, 0, 1)
        return torch.tensor(image)

    def _transform(self, img, addmean=True):
        if self.transform is not None:
            img = self.transform(image=img)['image']

        img = img.astype(np.float32)
        if addmean:
            img += self.mean
        return normalize_input(img)

### Model Architecture

In [3]:

def initialize_weights(net):
    for m in net.modules():
        try:
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        except Exception as e:
            # print(f'SKip layer {m}, {e}')
            pass

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(channels, out_channels,
            kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)
        self.ins_norm = nn.InstanceNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.2, True)
        initialize_weights(self)

    def forward(self, x):
        out = self.conv(x)
        out = self.ins_norm(out)
        out = self.activation(out)
        return out

class SeparableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, bias=False):
        super(SeparableConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3,
            stride=stride, padding=1, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(in_channels, out_channels,
            kernel_size=1, stride=1, bias=bias)
        # self.pad =
        self.ins_norm1 = nn.InstanceNorm2d(in_channels)
        self.activation1 = nn.LeakyReLU(0.2, True)
        self.ins_norm2 = nn.InstanceNorm2d(out_channels)
        self.activation2 = nn.LeakyReLU(0.2, True)
        initialize_weights(self)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.ins_norm1(out)
        out = self.activation1(out)
        out = self.pointwise(out)
        out = self.ins_norm2(out)
        return self.activation2(out)

class DownConv(nn.Module):
    def __init__(self, channels, bias=False):
        super(DownConv, self).__init__()
        self.conv1 = SeparableConv2D(channels, channels, stride=2, bias=bias)
        self.conv2 = SeparableConv2D(channels, channels, stride=1, bias=bias)

    def forward(self, x):
        out1 = self.conv1(x)
        out2 = F.interpolate(x, scale_factor=0.5, mode='bilinear')
        out2 = self.conv2(out2)
        return out1 + out2

class UpConv(nn.Module):
    def __init__(self, channels, bias=False):
        super(UpConv, self).__init__()
        self.conv = SeparableConv2D(channels, channels, stride=1, bias=bias)

    def forward(self, x):
        out = F.interpolate(x, scale_factor=2.0, mode='bilinear')
        out = self.conv(out)
        return out

class InvertedResBlock(nn.Module):
    def __init__(self, channels=256, out_channels=256, expand_ratio=2, bias=False):
        super(InvertedResBlock, self).__init__()
        bottleneck_dim = round(expand_ratio * channels)
        self.conv_block = ConvBlock(channels, bottleneck_dim, kernel_size=1, stride=1, padding=0, bias=bias)
        self.depthwise_conv = nn.Conv2d(bottleneck_dim, bottleneck_dim,
            kernel_size=3, groups=bottleneck_dim, stride=1, padding=1, bias=bias)
        self.conv = nn.Conv2d(bottleneck_dim, out_channels,
            kernel_size=1, stride=1, bias=bias)
        self.ins_norm1 = nn.InstanceNorm2d(bottleneck_dim)
        self.ins_norm2 = nn.InstanceNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.2, True)
        initialize_weights(self)

    def forward(self, x):
        out = self.conv_block(x)
        out = self.depthwise_conv(out)
        out = self.ins_norm1(out)
        out = self.activation(out)
        out = self.conv(out)
        out = self.ins_norm2(out)
        return out + x

In [5]:
class Generator(nn.Module):
    def __init__(self, dataset=''):
        super(Generator, self).__init__()
        self.name = f'generator_{dataset}'
        bias = False

        self.encode_blocks = nn.Sequential(
            ConvBlock(3, 64, bias=bias),
            ConvBlock(64, 128, bias=bias),
            DownConv(128, bias=bias),
            ConvBlock(128, 128, bias=bias),
            SeparableConv2D(128, 256, bias=bias),
            DownConv(256, bias=bias),
            ConvBlock(256, 256, bias=bias),
        )

        self.res_blocks = nn.Sequential(
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
            InvertedResBlock(256, 256, bias=bias),
        )

        self.decode_blocks = nn.Sequential(
            ConvBlock(256, 128, bias=bias),
            UpConv(128, bias=bias),
            SeparableConv2D(128, 128, bias=bias),
            ConvBlock(128, 128, bias=bias),
            UpConv(128, bias=bias),
            ConvBlock(128, 64, bias=bias),
            ConvBlock(64, 64, bias=bias),
            nn.Conv2d(64, 3, kernel_size=1, stride=1, padding=0, bias=bias),
            nn.Tanh(),
        )
        initialize_weights(self)

    def forward(self, x):
        out = self.encode_blocks(x)
        out = self.res_blocks(out)
        img = self.decode_blocks(out)
        return img

In [6]:
class Discriminator(nn.Module):
    def __init__(self, dataset, args):
        super(Discriminator, self).__init__()
        self.name = f"discriminator_{dataset}"
        self.bias = False
        channels = 32

        layers = [
            nn.Conv2d(3, channels, kernel_size=3, stride=1, padding=1, bias=self.bias),
            nn.LeakyReLU(0.2, True)
        ]

        for i in range(args['d_layers']):
            layers += [
                nn.Conv2d(channels, channels * 2, kernel_size=3, stride=2, padding=1, bias=self.bias),
                nn.LeakyReLU(0.2, True),
                nn.Conv2d(channels * 2, channels * 4, kernel_size=3, stride=1, padding=1, bias=self.bias),
                nn.InstanceNorm2d(channels * 4),
                nn.LeakyReLU(0.2, True),
            ]
            channels *= 4

        layers += [
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=self.bias),
            nn.InstanceNorm2d(channels),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(channels, 1, kernel_size=3, stride=1, padding=1, bias=self.bias),
        ]

        if args['use_sn']:
            for i in range(len(layers)):
                if isinstance(layers[i], nn.Conv2d):
                    layers[i] = spectral_norm(layers[i])

        self.discriminate = nn.Sequential(*layers)
        initialize_weights(self)

    def forward(self, img):
        return self.discriminate(img)

In [7]:
vgg_mean = torch.tensor([0.485, 0.456, 0.406]).float()
vgg_std = torch.tensor([0.229, 0.224, 0.225]).float()

if torch.cuda.is_available():
    vgg_std = vgg_std.cuda()
    vgg_mean = vgg_mean.cuda()

class Vgg19(nn.Module):
    def __init__(self):
        super(Vgg19, self).__init__()
        self.vgg19 = self.get_vgg19().eval()
        self.mean = vgg_mean.view(-1, 1 ,1)
        self.std = vgg_std.view(-1, 1, 1)

    def forward(self, x):
        return self.vgg19(self.normalize_vgg(x))

    @staticmethod
    def get_vgg19(last_layer='conv4_4'):
        vgg = models.vgg19(weights='IMAGENET1K_V1').features
        model_list = []
        i = 0
        j = 1
        for layer in vgg.children():
            if isinstance(layer, nn.MaxPool2d):
                i = 0
                j += 1
            elif isinstance(layer, nn.Conv2d):
                i += 1
            name = f'conv{j}_{i}'
            if name == last_layer:
                model_list.append(layer)
                break
            model_list.append(layer)
        model = nn.Sequential(*model_list)
        return model


    def normalize_vgg(self, image):
        '''
        Expect input in range -1 1
        '''
        image = (image + 1.0) / 2.0
        return (image - self.mean) / self.std

### Utils

In [8]:
import torch
import gc
import os
import torch.nn as nn
import urllib.request
import cv2
from tqdm import tqdm

HTTP_PREFIXES = [
    'http',
    'data:image/jpeg',
]

SUPPORT_WEIGHTS = {
    'hayao',
    'shinkai',
}

ASSET_HOST = 'https://github.com/ptran1203/pytorch-animeGAN/releases/download/v1.0'

def read_image(path):
    """
    Read image from given path
    """
    if any(path.startswith(p) for p in HTTP_PREFIXES):
        urllib.request.urlretrieve(path, "temp.jpg")
        path = "temp.jpg"
    return cv2.imread(path)[: ,: ,::-1]


def save_checkpoint(model, optimizer, epoch, args, posfix=''):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
    }
    path = os.path.join(args['checkpoint_dir'], f'{model.name}{posfix}.pth')
    torch.save(checkpoint, path)


def load_checkpoint(model, optimizer, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, optimizer, path)


def load_weight(model, optimizer, weight):
    if weight.lower() in SUPPORT_WEIGHTS:
        weight = _download_weight(weight)

    checkpoint = torch.load(weight,  map_location='cuda:0') if torch.cuda.is_available() else \
        torch.load(weight,  map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'], strict=True)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    del checkpoint
    torch.cuda.empty_cache()
    gc.collect()
    return epoch

def initialize_weights(net):
    for m in net.modules():
        try:
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        except Exception as e:
            # print(f'SKip layer {m}, {e}')
            pass

def set_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

class DownloadProgressBar(tqdm):
    '''
    https://stackoverflow.com/questions/15644964/python-progress-bar-and-downloads
    '''
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def _download_weight(weight):
    '''
    Download weight and save to local file
    '''
    filename = f'generator_{weight.lower()}.pth'
    os.makedirs('.cache', exist_ok=True)
    url = f'{ASSET_HOST}/{filename}'
    save_path = f'.cache/{filename}'
    if os.path.isfile(save_path):
        return save_path
    desc = f'Downloading {url} to {save_path}'
    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=desc) as t:
        urllib.request.urlretrieve(url, save_path, reporthook=t.update_to)
    return save_path

In [9]:
import torch
import cv2
import os
import numpy as np
from tqdm import tqdm

_rgb_to_yuv_kernel = torch.tensor([
    [0.299, -0.14714119, 0.61497538],
    [0.587, -0.28886916, -0.51496512],
    [0.114, 0.43601035, -0.10001026]
]).float()

if torch.cuda.is_available():
    _rgb_to_yuv_kernel = _rgb_to_yuv_kernel.cuda()

def gram(input):
    """
    Calculate Gram Matrix
    https://pytorch.org/tutorials/advanced/neural_style_tutorial.html#style-loss
    """
    b, c, w, h = input.size()
    x = input.view(b * c, w * h)
    G = torch.mm(x, x.T)
    # normalize by total elements
    return G.div(b * c * w * h)

def rgb_to_yuv(image):
    '''
    https://en.wikipedia.org/wiki/YUV
    output: Image of shape (H, W, C) (channel last)
    '''
    # -1 1 -> 0 1
    image = (image + 1.0) / 2.0
    yuv_img = torch.tensordot(
        image,
        _rgb_to_yuv_kernel,
        dims=([image.ndim - 3], [0]))
    return yuv_img


def divisible(dim):
    '''
    Make width and height divisible by 32
    '''
    width, height = dim
    return width - (width % 32), height - (height % 32)

def resize_image(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    h, w = image.shape[:2]
    if width and height:
        return cv2.resize(image, divisible((width, height)),  interpolation=inter)
    if width is None and height is None:
        return cv2.resize(image, divisible((w, h)),  interpolation=inter)
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    return cv2.resize(image, divisible(dim), interpolation=inter)


def normalize_input(images):
    '''
    [0, 255] -> [-1, 1]
    '''
    return images / 127.5 - 1.0


def denormalize_input(images, dtype=None):
    '''
    [-1, 1] -> [0, 255]
    '''
    images = images * 127.5 + 127.5
    if dtype is not None:
        if isinstance(images, torch.Tensor):
            images = images.type(dtype)
        else:
            # numpy.ndarray
            images = images.astype(dtype)

    return images


def compute_data_mean(data_folder):
    if not os.path.exists(data_folder):
        raise FileNotFoundError(f'Folder {data_folder} does not exits')
    image_files = os.listdir(data_folder)
    total = np.zeros(3)
    print(f"Compute mean (R, G, B) from {len(image_files)} images")
    for img_file in tqdm(image_files):
        path = os.path.join(data_folder, img_file)
        image = cv2.imread(path)
        total += image.mean(axis=(0, 1))
    channel_mean = total / len(image_files)
    mean = np.mean(channel_mean)
    return mean - channel_mean[...,::-1]  # Convert to BGR for training

### Loss

In [10]:
class ColorLoss(nn.Module):
    def __init__(self):
        super(ColorLoss, self).__init__()
        self.l1 = nn.L1Loss()
        self.huber = nn.SmoothL1Loss()

    def forward(self, image, image_g):
        image = rgb_to_yuv(image)
        image_g = rgb_to_yuv(image_g)

        return (self.l1(image[:, :, :, 0], image_g[:, :, :, 0]) +
                self.huber(image[:, :, :, 1], image_g[:, :, :, 1]) +
                self.huber(image[:, :, :, 2], image_g[:, :, :, 2]))


class AnimeGanLoss:
    def __init__(self, args):
        self.content_loss = nn.L1Loss().to(args['device'])
        self.gram_loss = nn.L1Loss().to(args['device'])
        self.color_loss = ColorLoss().to(args['device'])
        self.wadvg = args['wadvg']
        self.wadvd = args['wadvd']
        self.wcon = args['wcon']
        self.wgra = args['wgra']
        self.wcol = args['wcol']
        self.vgg19 = Vgg19().to(args['device']).eval()
        self.adv_type = args['gan_loss']
        self.bce_loss = nn.BCELoss()

    def compute_loss_G(self, fake_img, img, fake_logit, anime_gray):
        '''
        Compute loss for Generator

        @Arugments:
            - fake_img: generated image
            - img: image
            - fake_logit: output of Discriminator given fake image
            - anime_gray: grayscale of anime image

        @Returns:
            loss
        '''
        fake_feat = self.vgg19(fake_img)
        anime_feat = self.vgg19(anime_gray)
        img_feat = self.vgg19(img).detach()

        return [
            self.wadvg * self.adv_loss_g(fake_logit),
            self.wcon * self.content_loss(img_feat, fake_feat),
            self.wgra * self.gram_loss(gram(anime_feat), gram(fake_feat)),
            self.wcol * self.color_loss(img, fake_img),
        ]

    def compute_loss_D(self, fake_img_d, real_anime_d, real_anime_gray_d, real_anime_smooth_gray_d):
        return self.wadvd * (
            self.adv_loss_d_real(real_anime_d) +
            self.adv_loss_d_fake(fake_img_d) +
            self.adv_loss_d_fake(real_anime_gray_d) +
            0.2 * self.adv_loss_d_fake(real_anime_smooth_gray_d)
        )


    def content_loss_vgg(self, image, recontruction):
        feat = self.vgg19(image)
        re_feat = self.vgg19(recontruction)

        return self.content_loss(feat, re_feat)

    def adv_loss_d_real(self, pred):
        if self.adv_type == 'hinge':
            return torch.mean(F.relu(1.0 - pred))

        elif self.adv_type == 'lsgan':
            return torch.mean(torch.square(pred - 1.0))

        elif self.adv_type == 'normal':
            return self.bce_loss(pred, torch.ones_like(pred))

        raise ValueError(f'Do not support loss type {self.adv_type}')

    def adv_loss_d_fake(self, pred):
        if self.adv_type == 'hinge':
            return torch.mean(F.relu(1.0 + pred))

        elif self.adv_type == 'lsgan':
            return torch.mean(torch.square(pred))

        elif self.adv_type == 'normal':
            return self.bce_loss(pred, torch.zeros_like(pred))

        raise ValueError(f'Do not support loss type {self.adv_type}')


    def adv_loss_g(self, pred):
        if self.adv_type == 'hinge':
            return -torch.mean(pred)

        elif self.adv_type == 'lsgan':
            return torch.mean(torch.square(pred - 1.0))

        elif self.adv_type == 'normal':
            return self.bce_loss(pred, torch.zeros_like(pred))

        raise ValueError(f'Do not support loss type {self.adv_type}')


class LossSummary:
    def __init__(self):
        self.reset()

    def reset(self):
        self.loss_g_adv = []
        self.loss_content = []
        self.loss_gram = []
        self.loss_color = []
        self.loss_d_adv = []

    def update_loss_G(self, adv, gram, color, content):
        self.loss_g_adv.append(adv.cpu().detach().numpy())
        self.loss_gram.append(gram.cpu().detach().numpy())
        self.loss_color.append(color.cpu().detach().numpy())
        self.loss_content.append(content.cpu().detach().numpy())

    def update_loss_D(self, loss):
        self.loss_d_adv.append(loss.cpu().detach().numpy())

    def avg_loss_G(self):
        return (
            self._avg(self.loss_g_adv),
            self._avg(self.loss_gram),
            self._avg(self.loss_color),
            self._avg(self.loss_content),
        )

    def avg_loss_D(self):
        return self._avg(self.loss_d_adv)


    @staticmethod
    def _avg(losses):
        return sum(losses) / len(losses)


In [11]:
def gaussian_noise():
    return torch.normal(torch.tensor(0.0), torch.tensor(0.1))

def checkDir(args):
    if not os.path.exists(args['checkpoint_dir']):
        print(f"* {args['checkpoint_dir']} does not exist, creating...")
        os.makedirs(args['checkpoint_dir'])
    if not os.path.exists(args['save_image_dir']):
        print(f"* {args['save_image_dir']} does not exist, creating...")
        os.makedirs(args['save_image_dir'])

def collate_fn(batch):
    img, anime, anime_gray, anime_smt_gray = zip(*batch)
    return (
        torch.stack(img, 0),
        torch.stack(anime, 0),
        torch.stack(anime_gray, 0),
        torch.stack(anime_smt_gray, 0),
    )

def save_samples(generator, loader, args, max_imgs=2, subname='gen'):
    '''
    Generate and save images
    '''
    generator.eval()
    max_iter = (max_imgs // args['batch_size']) + 1
    fake_imgs = []
    for i, (img, *_) in enumerate(loader):
        with torch.no_grad():
            fake_img = generator(img.cuda())
            fake_img = fake_img.detach().cpu().numpy()
            # Channel first -> channel last
            fake_img = fake_img.transpose(0, 2, 3, 1)
            fake_imgs.append(denormalize_input(fake_img, dtype=np.int16))
        if i + 1 == max_iter:
            break
    fake_imgs = np.concatenate(fake_imgs, axis=0)
    for i, img in enumerate(fake_imgs):
        save_path = os.path.join(args['save_image_dir'], f'{subname}_{i}.jpg')
        cv2.imwrite(save_path, img[..., ::-1])

def main(args):
    checkDir(args)
    G = Generator('Hayao').to(args['device'])
#     G = DataParallel(G)
    D = Discriminator('Hayao', args).to(args['device'])
#     D = DataParallel(D)

    loss_tracker = LossSummary()
    loss_fn = AnimeGanLoss(args)
    optimizer_g = optim.Adam(G.parameters(), lr=args['lr_g'], betas=(0.5, 0.999))
    optimizer_d = optim.Adam(D.parameters(), lr=args['lr_d'], betas=(0.5, 0.999))

    data_loader = DataLoader(
        AnimeDataSet(args),
        batch_size=args['batch_size'],
        pin_memory=True,
        shuffle=True,
        collate_fn=collate_fn,
    )

    start_e = 0
    if args['resume'] == 'GD':
        # Load G and D
        try:
            start_e = load_checkpoint(G, optimizer_g, '/kaggle/working/checkpoints')+1
            print("G weight loaded")
            load_checkpoint(D, optimizer_d, '/kaggle/working/checkpoints')
            print("D weight loaded")
        except Exception as e:
            print('Could not load checkpoint, train from scratch', e)
    elif args['resume'] == 'G':
        # Load G only
        try:
            start_e = load_checkpoint(G, optimizer_g, args['checkpoint_dir'], posfix='_init')+1
        except Exception as e:
            print('Could not load G init checkpoint, train from scratch', e)

    for e in range(start_e, args['epochs']):
        print(f"Epoch {e}/{args['epochs']}")
        bar = tqdm(data_loader)
        G.train()
        init_losses = []
        if e < args['init_epochs']:
            # Train with content loss only
            set_lr(optimizer_g, args['init_lr'])
            for img, *_ in bar:
                img = img.to(args['device'])
                optimizer_g.zero_grad()

                fake_img = G(img)
                loss = loss_fn.content_loss_vgg(img, fake_img)
                loss.backward()
                optimizer_g.step()

                init_losses.append(loss.cpu().detach().numpy())
                avg_content_loss = sum(init_losses) / len(init_losses)
                bar.set_description(f'[Init Training G] content loss: {avg_content_loss:2f}')

            set_lr(optimizer_g, args['lr_g'])
            save_checkpoint(G, optimizer_g, e, args, posfix='_init')
            save_samples(G, data_loader, args, subname='initg')
            continue

        loss_tracker.reset()
        for i, (img, anime, anime_gray, anime_smt_gray) in enumerate(bar):
            img = img.to(args['device'])
            anime = anime.to(args['device'])
            anime_gray = anime_gray.to(args['device'])
            anime_smt_gray = anime_smt_gray.to(args['device'])

#             # ---------------- TRAIN D ---------------- #
#             optimizer_d.zero_grad()
#             fake_img = G(img).detach()
#             # Add some Gaussian noise to images before feeding to D
#             if args['d_noise']:
#                 fake_img += gaussian_noise()
#                 anime += gaussian_noise()
#                 anime_gray += gaussian_noise()
#                 anime_smt_gray += gaussian_noise()
#             fake_d = D(fake_img)
#             real_anime_d = D(anime)
#             real_anime_gray_d = D(anime_gray)
#             real_anime_smt_gray_d = D(anime_smt_gray)
#             # Compute Loss
#             loss_d = loss_fn.compute_loss_D(
#                 fake_d, real_anime_d, real_anime_gray_d, real_anime_smt_gray_d)
#             # Backward
#             loss_d.backward()
#             optimizer_d.step()
#             loss_tracker.update_loss_D(loss_d)

            # ---------------- TRAIN G ---------------- #
            optimizer_g.zero_grad()
            fake_img = G(img)
            fake_d = D(fake_img)
            # Compute Loss
            adv_loss, con_loss, gra_loss, col_loss = loss_fn.compute_loss_G(
                fake_img, img, fake_d, anime_gray)
            loss_g = adv_loss + con_loss + gra_loss + col_loss
            # Backward
            loss_g.backward()
            optimizer_g.step()
            loss_tracker.update_loss_G(adv_loss, gra_loss, col_loss, con_loss)
            
            # ---------------- TRAIN D ---------------- #
            if i%15 == 0:
                optimizer_d.zero_grad()
                fake_img = G(img).detach()
                # Add some Gaussian noise to images before feeding to D
                if args['d_noise']:
                    fake_img += gaussian_noise()
                    anime += gaussian_noise()
                    anime_gray += gaussian_noise()
                    anime_smt_gray += gaussian_noise()
                fake_d = D(fake_img)
                real_anime_d = D(anime)
                real_anime_gray_d = D(anime_gray)
                real_anime_smt_gray_d = D(anime_smt_gray)
                # Compute Loss
                loss_d = loss_fn.compute_loss_D(
                    fake_d, real_anime_d, real_anime_gray_d, real_anime_smt_gray_d)
                # Backward
                loss_d.backward()
                optimizer_d.step()
                loss_tracker.update_loss_D(loss_d)

            avg_adv, avg_gram, avg_color, avg_content = loss_tracker.avg_loss_G()
            avg_adv_d = loss_tracker.avg_loss_D()
            bar.set_description(f'loss G: adv {avg_adv:2f} con {avg_content:2f} gram {avg_gram:2f} color {avg_color:2f} / loss D: {avg_adv_d:2f}')
            
            
        if e % args['save_interval'] == 0:
            save_checkpoint(G, optimizer_g, e, args)
            save_checkpoint(D, optimizer_d, e, args)
            save_samples(G, data_loader, args)

In [12]:
json_config = {
  "dataset": "/kaggle/input/animedataset/Hayao",
  "data_dir": "/kaggle/input/animedataset",
  "epochs": 100,
  "init_epochs": 5,
  "batch_size": 8,
  "device": "cuda",
  "checkpoint_dir": "/kaggle/working/checkpoints",
  "save_image_dir": "/kaggle/working/samples",
  "lr_g": 8e-5,
  "lr_d": 16e-5,
  "init_lr": 1e-3,
  "wadvg": 30,
  "wadvd": 30,
  "wcon": 1.5,
  "wgra": 3,
  "wcol": 30,
  "resume": "GD",
  "save_interval": 1,
  "d_layers": 3,
  "use_sn": "True",
  "gan_loss": "lsgan",
  "d_noise": "True"
}

In [ ]:
main(json_config)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 256MB/s]  


Compute mean (R, G, B) from 1792 images


100%|██████████| 1792/1792 [00:14<00:00, 127.03it/s]


Mean(B, G, R) of /kaggle/input/anime-dataset/Hayao are [-4.4346958  -8.66591597 13.10061177]
Dataset: real 6656 style 1792, smooth 1792
G weight loaded
D weight loaded
Epoch 27/100


loss G: adv 9.973940 con 3.640771 gram 0.021225 color 1.734808 / loss D: 0.299507: 100%|██████████| 1110/1110 [31:05<00:00,  1.68s/it]


...

loss G: adv 9.951957 con 3.539820 gram 0.021132 color 1.667478 / loss D: 0.289328: 100%|██████████| 1110/1110 [30:24<00:00,  1.64s/it]


Epoch 30/100


loss G: adv 9.981235 con 3.449651 gram 0.021264 color 1.623905 / loss D: 0.232718: 100%|██████████| 1110/1110 [30:24<00:00,  1.64s/it]


Epoch 31/100


loss G: adv 9.972983 con 3.454667 gram 0.021285 color 1.609291 / loss D: 0.264187:  67%|██████▋   | 748/1110 [20:29<09:56,  1.65s/it]

In [13]:
os.listdir('/kaggle/working/checkpoints')

['generator_Hayao.pth', 'discriminator_Hayao.pth', 'generator_Hayao_init.pth']

In [14]:
!zip -r /kaggle/working/checkpoints-51.zip /kaggle/working/checkpoints

  adding: kaggle/working/checkpoints/ (stored 0%)
  adding: kaggle/working/checkpoints/generator_Hayao.pth (deflated 8%)
  adding: kaggle/working/checkpoints/discriminator_Hayao.pth (deflated 8%)
  adding: kaggle/working/checkpoints/generator_Hayao_init.pth (deflated 9%)


In [15]:
from IPython.display import FileLink

In [16]:
%cd /kaggle/working

/kaggle/working


In [17]:
FileLink(r'checkpoints-51.zip')

/kaggle/working/checkpoints-40.zip

In [13]:
def input_transform(image_path):
    image = cv2.imread(image_path)[:,:,::-1]
    image = image.astype(np.float32)
    image = normalize_input(image)
    image = image.transpose(2, 0, 1)
    return torch.tensor(image)

In [ ]:
test_generator = Generator(args.dataset)
checkpoint = torch.load(weight,  map_location='cuda:0') if torch.cuda.is_available() else \
    torch.load(weight,  map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
epoch = checkpoint['epoch']
del checkpoint
torch.cuda.empty_cache()
gc.collect()
generator.eval()
max_iter = (max_imgs // args['batch_size']) + 1
fake_imgs = []
for i, (img, *_) in enumerate(loader):
    with torch.no_grad():
        fake_img = generator(img.cuda())
        fake_img = fake_img.detach().cpu().numpy()
        # Channel first -> channel last
        fake_img = fake_img.transpose(0, 2, 3, 1)
        fake_imgs.append(denormalize_input(fake_img, dtype=np.int16))
    if i + 1 == max_iter:
        break
fake_imgs = np.concatenate(fake_imgs, axis=0)
for i, img in enumerate(fake_imgs):
    save_path = os.path.join(args['save_image_dir'], f'{subname}_{i}.jpg')
    cv2.imwrite(save_path, img[..., ::-1])